In [1]:
import org.apache.hadoop.conf.Configuration
import org.apache.hadoop.io.{LongWritable, Text}
import org.apache.hadoop.mapreduce.lib.input.TextInputFormat

In [65]:
val spark = SparkSession
                .builder
                .master("local[*]")
                .appName("SparkSQL")
                .config("spark.cassandra.connection.host", "127.0.0.1")
                .getOrCreate()

import spark.implicits._

// Reading the file from disk
val conf = new Configuration
conf.set("textinputformat.record.delimiter", "WARC/1.0")
val dataset = spark
                .sparkContext
                .newAPIHadoopFile(
                    "file:///home/hrabo/Documents/skola/data-intensive-computing/project/data/CC-MAIN-20180918130631-20180918150631-00000.warc.wet",
//                     "./data/WET/CC-MAIN-20180918130631-20180918150631-00000.warc.wet",
                    classOf[TextInputFormat], 
                    classOf[LongWritable], 
                    classOf[Text], 
                    conf
                )

val tempdata = dataset.map(x=>x._2.toString)      

val data = tempdata.mapPartitionsWithIndex {  // Drop invalid records
                    (idx, iter) => if (idx == 0) iter.drop(2) else iter 
                  }

// data.take(1).foreach(println)

spark = org.apache.spark.sql.SparkSession@50280db1
conf = Configuration: core-default.xml, core-site.xml, mapred-default.xml, mapred-site.xml, yarn-default.xml, yarn-site.xml, hdfs-default.xml, hdfs-site.xml
dataset = file:///home/hrabo/Documents/skola/data-intensive-computing/project/data/CC-MAIN-20180918130631-20180918150631-00000.warc.wet NewHadoopRDD[75] at newAPIHadoopFile at <console>:174
tempdata = MapPartitionsRDD[76] at map at <console>:183
data = MapPartitionsRDD[77] at mapPartitionsWithIndex at <console>:185


lastException: Throwable = null


MapPartitionsRDD[77] at mapPartitionsWithIndex at <console>:185

In [63]:
// data.take(1).foreach(println)

Name: Unknown Error
Message: <console>:160: error: value config is not a member of org.apache.spark.sql.SparkSession
       spark.config
             ^

StackTrace: 

In [66]:
// List of tuples containing (metadata, file_content)
import java.net.URL

val uriDataPairs = data.map(record => (record.split("\r\n\r\n")))
                       .map(dataItemAsList => {
                            // Pairs of (Property, Value) from the meta data
                            val keyValuePairs = dataItemAsList(0).split("\r\n").filter(_ != "").map(line => {
                                val splitList = line.split(": ")
                                ((splitList(0), splitList(1)))
                            }) 
                            
                            // Selecting the first (and only) record and the value from that
                            val targetURL = keyValuePairs.filter(_._1 == "WARC-Target-URI")(0)._2
                            
                            ((targetURL,dataItemAsList(1)))
                       })//.take(2)(1)._1

// Create DF
val pageInfo = uriDataPairs.map(item => {
    val host = new URL(item._1).getHost
    val domainParts = host.replaceAll("""([^\.]*+\.)""", "") // Replace everything except top domain
    (item._1, domainParts, item._2)
})//.take(1)

// val topDomainCom = pageInfo.filter(x)

val pageInfoDF = pageInfo.toDF("uri", "topDomain", "text")
pageInfoDF.show

+--------------------+---------+--------------------+
|                 uri|topDomain|                text|
+--------------------+---------+--------------------+
|http://0-50.ru/ne...|       ru|Новости 0-50.ru |...|
|http://000elham00...|      com|تبلیغات	
مسخره - ...|
|http://0105335908...|       kr|맛집? 많은분 들이 이곳을 이용...|
|http://01ninchisy...|      net|認知症ケアマニュアル: アルツハイ...|
|http://01xxxx.com...|      com|正在播放 5-鬼畜老师强暴学生妹[...|
|http://023pifu.co...|      com|皮肤病 / 荨麻疹_重庆皮肤病医院...|
|http://03e98e7.ne...|      com|Amicus Brief FAQs...|
|http://0411auto.c...|      com|2017款宝马X5 xDrive3...|
|http://0412hitomi...|      com|こら！ぼたち！ ダッシュするときも...|
|http://080ek21.co...|      com|網路聊天聯盟
回 首 頁 │ 點數...|
|http://09irk.ru/r...|       ru|Мангал | Леко мно...|
|http://0ino0.beon...|       ru|Sasori в дневнике...|
|http://0we.kunlun...|      com|Welcome 0we.kunlu...|
|http://1-800-opti...|      com|Unisex Patches
1-...|
|http://1-tech.com...|       ua|Купить CABHD-360-...|
|http://10-carat-d...|      

uriDataPairs = MapPartitionsRDD[79] at map at <console>:178
pageInfo = MapPartitionsRDD[80] at map at <console>:192
pageInfoDF = [uri: string, topDomain: string ... 1 more field]


[uri: string, topDomain: string ... 1 more field]

In [67]:
import org.apache.spark.sql.functions._ 

val extractKeyWords = udf[Array[(String, Int)], String](input => {
    val stopWords = Set("i","me","my","myself","we","our","ours","ourselves","you","your","yours","yourself","yourselves","he","him","his","himself","she","her","hers","herself","it","its","itself","they","them","their","theirs","themselves","what","which","who","whom","this","that","these","those","am","is","are","was","were","be","been","being","have","has","had","having","do","does","did","doing","a","an","the","and","but","if","or","because","as","until","while","of","at","by","for","with","about","against","between","into","through","during","before","after","above","below","to","from","up","down","in","out","on","off","over","under","again","further","then","once","here","there","when","where","why","how","all","any","both","each","few","more","most","other","some","such","no","nor","not","only","own","same","so","than","too","very","s","t","can","will","just","don","should","now")
    
    input
      .split("[\\s|\n]")
      .map(s => s.replaceAll("""[^\x00-\x7F]""", "")) // We only manage ascii chars
      .map(s => s.replaceAll("""^\p{Punct}*$""", "")) // No words with only punctuation
      .map(s => s.replaceAll("""\p{Punct}$""", ""))   // No trailing punctuations
      .filter(_ != "")                                // No empty words
      .map(_.toLowerCase)
      .filter(!stopWords.contains(_))                 // Remove stop-words
      .groupBy(identity).mapValues(_.size).toArray
      .sortBy(- _._2)
})


val keywordDF = pageInfoDF.withColumn(
    "keywords",
    extractKeyWords(col("text"))
  )

keywordDF
//     .select("keywords")
    .show()


+--------------------+---------+--------------------+--------------------+
|                 uri|topDomain|                text|            keywords|
+--------------------+---------+--------------------+--------------------+
|http://0-50.ru/ne...|       ru|Новости 0-50.ru |...|[[2, 13], [1:1, 4...|
|http://000elham00...|      com|تبلیغات	
مسخره - ...|[[1396, 15], [139...|
|http://0105335908...|       kr|맛집? 많은분 들이 이곳을 이용...|[[01053359081.kte...|
|http://01ninchisy...|      net|認知症ケアマニュアル: アルツハイ...|[[(7, 2], [(5, 2]...|
|http://01xxxx.com...|      com|正在播放 5-鬼畜老师强暴学生妹[...|[[2017-02-16, 13]...|
|http://023pifu.co...|      com|皮肤病 / 荨麻疹_重庆皮肤病医院...|[[hospital, 3], [...|
|http://03e98e7.ne...|      com|Amicus Brief FAQs...|[[appellate, 4], ...|
|http://0411auto.c...|      com|2017款宝马X5 xDrive3...|[[xdrive35i, 3], ...|
|http://0412hitomi...|      com|こら！ぼたち！ ダッシュするときも...|[[(31, 15], [(30,...|
|http://080ek21.co...|      com|網路聊天聯盟
回 首 頁 │ 點數...|[[ut, 40], [0401,...|
|http://09irk.ru/r...|   

extractKeyWords = UserDefinedFunction(<function1>,ArrayType(StructType(StructField(_1,StringType,true), StructField(_2,IntegerType,false)),true),Some(List(StringType)))
keywordDF = [uri: string, topDomain: string ... 2 more fields]


[uri: string, topDomain: string ... 2 more fields]

In [68]:
// create one list with all keywords
// keywords distinct = 2017741 (.distinct.count)
// keywords total = 15 000 000
// total_nr_records = 39653
val keywordsExpandedDF = keywordDF
                            .select("keywords", "uri")
                            .withColumn("keywords", explode(($"keywords")))
                            .select("keywords._1", "keywords._2", "uri")
                            .withColumnRenamed("_1", "keyword")
                            .withColumnRenamed("_2", "keyword_weight")
keywordsExpandedDF.show


+---------+--------------+--------------------+
|  keyword|keyword_weight|                 uri|
+---------+--------------+--------------------+
|        2|            13|http://0-50.ru/ne...|
|      1:1|             4|http://0-50.ru/ne...|
|       15|             3|http://0-50.ru/ne...|
|      1:0|             3|http://0-50.ru/ne...|
|      0:1|             3|http://0-50.ru/ne...|
|      1:2|             3|http://0-50.ru/ne...|
|     2017|             3|http://0-50.ru/ne...|
|      2:0|             2|http://0-50.ru/ne...|
|       22|             2|http://0-50.ru/ne...|
|    -2114|             2|http://0-50.ru/ne...|
|      0:2|             2|http://0-50.ru/ne...|
|        5|             2|http://0-50.ru/ne...|
|      2:1|             2|http://0-50.ru/ne...|
|    -2012|             2|http://0-50.ru/ne...|
|      0:0|             2|http://0-50.ru/ne...|
|        7|             2|http://0-50.ru/ne...|
|      rss|             2|http://0-50.ru/ne...|
|     2014|             1|http://0-50.ru

keywordsExpandedDF = [keyword: string, keyword_weight: int ... 1 more field]


[keyword: string, keyword_weight: int ... 1 more field]

In [69]:
var reverseIndex = keywordsExpandedDF
                        .groupBy($"keyword")
                        .agg(
                            collect_list($"uri").alias("uri_list"), 
                            collect_list($"keyword_weight").alias("keyword_weights")
                            ) 
                        // We can probably collect as struct collect_list(struct($"uri", $"keyword_weight")).as("set")
                        // This might however lead to problems: https://stackoverflow.com/questions/31864744/spark-dataframes-groupby-into-list
                        
// Showing here takes 1 minute
// reducedKeywords.show() 


reverseIndex = [keyword: string, uri_list: array<string> ... 1 more field]


[keyword: string, uri_list: array<string> ... 1 more field]

In [12]:
// val hellos = reducedKeywords.filter($"keyword" === "hello").select($"uri_list").cache
// hellos.show()

// // Numer of pages using the word "hello" = 756
// println(hellos.first.getList(0).size)


Name: Unknown Error
Message: <console>:79: error: overloaded method value groupBy with alternatives:
  (col1: String,cols: String*)org.apache.spark.sql.RelationalGroupedDataset <and>
  (cols: org.apache.spark.sql.Column*)org.apache.spark.sql.RelationalGroupedDataset
 cannot be applied to (org.apache.spark.sql.ColumnName)
                               .groupBy($"keyword")
                                ^

StackTrace: 

In [19]:
// Download cassandra dependencies
%AddJar https://repo1.maven.org/maven2/com/datastax/spark/spark-cassandra-connector_2.11/2.0.9/spark-cassandra-connector_2.11-2.0.9.jar -f
%AddJar https://repo1.maven.org/maven2/com/datastax/cassandra/cassandra-driver-core/3.0.0/cassandra-driver-core-3.0.0.jar -f


Starting download from https://repo1.maven.org/maven2/com/datastax/spark/spark-cassandra-connector_2.11/2.0.9/spark-cassandra-connector_2.11-2.0.9.jar
Finished download of spark-cassandra-connector_2.11-2.0.9.jar
Starting download from https://repo1.maven.org/maven2/com/datastax/cassandra/cassandra-driver-core/3.0.0/cassandra-driver-core-3.0.0.jar
Finished download of cassandra-driver-core-3.0.0.jar


In [70]:
// Import and connect to cassandra
import org.apache.spark.sql.cassandra._
import com.datastax.spark.connector._
import com.datastax.driver.core.{Session, Cluster, Host, Metadata}
import com.datastax.spark.connector.cql.CassandraConnectorConf
import com.datastax.spark.connector.rdd.ReadConf

// Setting up for cassandra
// spark.setCassandraConf(CassandraConnectorConf.KeepAliveMillisParam.option(10000))
// spark.setCassandraConf("localCluster", CassandraConnectorConf.ConnectionHostParam.option("127.0.0.1")) 
// spark.setCassandraConf("localCluster", "search", ReadConf.SplitSizeInMBParam.option(128))

val cluster = Cluster.builder().addContactPoint("127.0.0.1").build()
val session = cluster.connect()

cluster = com.datastax.driver.core.Cluster@773970c9
session = com.datastax.driver.core.SessionManager@70fa7c06


com.datastax.driver.core.SessionManager@70fa7c06

In [60]:
// Create tables
session.execute("CREATE KEYSPACE IF NOT EXISTS search WITH REPLICATION =" +
                "{'class': 'SimpleStrategy', 'replication_factor': 1};")
session.execute("CREATE TABLE IF NOT EXISTS search.keywords (keyword text PRIMARY KEY, links list<text>, occurences list<int>);")


session.close()

Name: com.datastax.driver.core.exceptions.NoHostAvailableException
Message: All host(s) tried for query failed (no host was tried)
StackTrace:   at com.datastax.driver.core.exceptions.NoHostAvailableException.copy(NoHostAvailableException.java:84)
  at com.datastax.driver.core.exceptions.NoHostAvailableException.copy(NoHostAvailableException.java:37)
  at com.datastax.driver.core.DriverThrowables.propagateCause(DriverThrowables.java:37)
  at com.datastax.driver.core.DefaultResultSetFuture.getUninterruptibly(DefaultResultSetFuture.java:245)
  at com.datastax.driver.core.AbstractSession.execute(AbstractSession.java:68)
  at com.datastax.driver.core.AbstractSession.execute(AbstractSession.java:43)
  ... 88 elided
Caused by: com.datastax.driver.core.exceptions.NoHostAvailableException: All host(s) tried for query failed (no host was tried)
  at com.datastax.driver.core.RequestHandler.reportNoMoreHosts(RequestHandler.java:211)
  at com.datastax.driver.core.RequestHandler.access$1000(Request

In [34]:
reverseIndex.rdd

lastException: Throwable = null


MapPartitionsRDD[71] at rdd at <console>:126

In [73]:
// Saving to cassandra

// connect to Cassandra and make a keyspace and table as explained in the document
// reverseIndex.rdd.saveToCassandra("search", "keywords",  SomeColumns("keyword", "links", "occurences"))
// reverseIndex.rdd.bulkSaveToCassandra("search", "keywords")

reverseIndex
    .write
    .format("org.apache.spark.sql.cassandra")
    .cassandraFormat("keywords", "search")
//     .options(Map("keyspace"->"search","table"->"keywords"))
//     .mode(SaveMode.Append)
    .save()

lastException = null


Name: java.lang.NoClassDefFoundError
Message: Could not initialize class org.apache.spark.sql.cassandra.DefaultSource$
StackTrace:   at org.apache.spark.sql.cassandra.package$DataFrameWriterWrapper$.cassandraFormat$extension1(package.scala:55)